# Run multiple rainfall events

In [1]:
from pathlib import Path
import mikeio
import mikeplus as mp

rainfall_timeseries = Path("../tests/testdata/Db/Sirius/Sirius_IDF_rainfall.dfs0")
db_path = Path("../tests/testdata//Db/Sirius/Sirius.sqlite")

C:\dev\mikepluspy\mikeplus\__init__.py:10: UserWarning: mikeio and mikeplus are both imported in the same process. There could be some conflicts. See docs for more info.
  _check_conflicts()


#### Explore rainfall boundary dfs0 file

In [2]:
ds = mikeio.read(rainfall_timeseries)
ds

<mikeio.Dataset>
dims: (time:22)
time: 2019-01-01 00:00:00 - 2019-01-02 00:00:00 (22 non-equidistant records)
geometry: GeometryUndefined()
items:
  0:   F=20 <Rainfall Intensity> (mm per hour) - 3
  1:   F=10 <Rainfall Intensity> (mm per hour) - 3
  2:   F=5 <Rainfall Intensity> (mm per hour) - 3
  3:   F=2 <Rainfall Intensity> (mm per hour) - 3
  4:   F=1 <Rainfall Intensity> (mm per hour) - 3
  5:   F=0.5 <Rainfall Intensity> (mm per hour) - 3
  6:   F=0.2 <Rainfall Intensity> (mm per hour) - 3
  7:   F=0.1 <Rainfall Intensity> (mm per hour) - 3
  8:   F=0.05 <Rainfall Intensity> (mm per hour) - 3

In [3]:
event_info = []
for item in ds.items:
    event_info.append(
        {
            "file_path": str(rainfall_timeseries.relative_to(db_path.parent)),
            "item_name": item.name,
            "data_type": repr(item.type),
        }
    )

event_info

[{'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=20',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=10',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=5',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=2',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=1',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.5',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.2',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.1',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.05',
  'data_type': 'Rainfall Intensity'}]

#### Create boundary conditions for each rainfall event

In [4]:
db = mp.open(db_path)
db

Database<'Sirius.sqlite'>

In [5]:
# No boundary conditions currently exist
msm_BBoundary = db.tables.msm_BBoundary
msm_BBoundary.get_muids()

[]

In [6]:
# Create a boundary condition for each event
msm_BBoundary = db.tables.msm_BBoundary
for event in event_info:
    msm_BBoundary.insert({
        msm_BBoundary.columns.MUID              : "Rainfall " + event["item_name"],
        msm_BBoundary.columns.VariationNo       : 3, # set temporal variation to timeseries
        msm_BBoundary.columns.FlowTypeNo        : 5, # set boundary condition type to Rainfall
        msm_BBoundary.columns.LoadTypeNo        : 1, # set load type to stormwater runoff
        msm_BBoundary.columns.TSConnection      : event["file_path"],
        msm_BBoundary.columns.TimeseriesName    : event["item_name"],
        msm_BBoundary.columns.DataTypeName      : event["data_type"],
        msm_BBoundary.columns.ApplyBoundaryNo   : 0,
    })

In [7]:
# Verify
def view_boundaries():
    return msm_BBoundary.select([
        msm_BBoundary.columns.VariationNo,
        msm_BBoundary.columns.FlowTypeNo,
        msm_BBoundary.columns.LoadTypeNo,
        msm_BBoundary.columns.TSConnection,
        msm_BBoundary.columns.TimeseriesName,
        msm_BBoundary.columns.DataTypeName,
        msm_BBoundary.columns.ApplyBoundaryNo,
    ]).to_dataframe()
view_boundaries()

,VariationNo,FlowTypeNo,LoadTypeNo,TSConnection,TimeseriesName,DataTypeName,ApplyBoundaryNo
Rainfall F=20,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=20,Rainfall Intensity,0
Rainfall F=10,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=10,Rainfall Intensity,0
Rainfall F=5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=5,Rainfall Intensity,0
Rainfall F=2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=2,Rainfall Intensity,0
Rainfall F=1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=1,Rainfall Intensity,0
Rainfall F=0.5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.5,Rainfall Intensity,0
Rainfall F=0.2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.2,Rainfall Intensity,0
Rainfall F=0.1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.1,Rainfall Intensity,0
Rainfall F=0.05,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.05,Rainfall Intensity,0


#### Create a scenario with a corresponding alternative for each rainfall

In [8]:
# We are in the base scenario
db.active_scenario

Scenario <Base>

In [9]:
# We'll create altenatives under the Loads and boundaries data alternative group
db.alternative_groups["Loads and boundaries data"].tables

['msm_BBoundary',
 'msm_LoadPoint',
 'msm_LoadPointConnection',
 'msm_BndGridRainWeights',
 'mrm_SHECouplings']

In [10]:
# The alternative for the active scenario
db.alternative_groups["Loads and boundaries data"].active

Alternative(Loads and boundaries data) <Base Alternative>

In [11]:
for boundary in msm_BBoundary.get_muids():

    # Create scenario
    scenario = db.scenarios.create(name=boundary, parent=db.active_scenario)

    # Create alternative
    alternative = db.alternative_groups["Loads and boundaries data"].create(
        name=boundary,
        parent=db.alternative_groups["Loads and boundaries data"].active
    )
    
    # Set alternative for new scenario
    scenario.set_alternative(alternative)

    # Activate the new scenario and update boundary
    db.active_scenario = scenario
    updated_muids = (
        msm_BBoundary
        .update({
            msm_BBoundary.columns.ApplyBoundaryNo: 1
        })
        .where(f"MUID = '{boundary}'")
        .execute() # don't forget to execute!
    )
    print(f"Updated boundary MUIDs: {updated_muids}")

    # Reset back to base scenario for next iteration
    db.active_scenario = db.scenarios.base


Updated boundary MUIDs: ['Rainfall  F=20']
Updated boundary MUIDs: ['Rainfall  F=10']
Updated boundary MUIDs: ['Rainfall  F=5']
Updated boundary MUIDs: ['Rainfall  F=2']
Updated boundary MUIDs: ['Rainfall  F=1']
Updated boundary MUIDs: ['Rainfall  F=0.5']
Updated boundary MUIDs: ['Rainfall  F=0.2']
Updated boundary MUIDs: ['Rainfall  F=0.1']
Updated boundary MUIDs: ['Rainfall  F=0.05']


In [12]:
# The base scenario should still have no boundaries applied
db.active_scenario = db.scenarios.base
view_boundaries()

,VariationNo,FlowTypeNo,LoadTypeNo,TSConnection,TimeseriesName,DataTypeName,ApplyBoundaryNo
Rainfall F=20,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=20,Rainfall Intensity,0
Rainfall F=10,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=10,Rainfall Intensity,0
Rainfall F=5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=5,Rainfall Intensity,0
Rainfall F=2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=2,Rainfall Intensity,0
Rainfall F=1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=1,Rainfall Intensity,0
Rainfall F=0.5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.5,Rainfall Intensity,0
Rainfall F=0.2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.2,Rainfall Intensity,0
Rainfall F=0.1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.1,Rainfall Intensity,0
Rainfall F=0.05,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.05,Rainfall Intensity,0


In [13]:
# Verify boundary is applied
db.active_scenario = db.scenarios.by_name("Rainfall  F=20")
view_boundaries()

,VariationNo,FlowTypeNo,LoadTypeNo,TSConnection,TimeseriesName,DataTypeName,ApplyBoundaryNo
Rainfall F=10,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=10,Rainfall Intensity,0
Rainfall F=5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=5,Rainfall Intensity,0
Rainfall F=2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=2,Rainfall Intensity,0
Rainfall F=1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=1,Rainfall Intensity,0
Rainfall F=0.5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.5,Rainfall Intensity,0
Rainfall F=0.2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.2,Rainfall Intensity,0
Rainfall F=0.1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.1,Rainfall Intensity,0
Rainfall F=0.05,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.05,Rainfall Intensity,0
Rainfall F=20,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=20,Rainfall Intensity,1


In [14]:
# Verify boundary is applied
db.active_scenario = db.scenarios.by_name("Rainfall  F=0.05")
view_boundaries()

,VariationNo,FlowTypeNo,LoadTypeNo,TSConnection,TimeseriesName,DataTypeName,ApplyBoundaryNo
Rainfall F=20,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=20,Rainfall Intensity,0
Rainfall F=10,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=10,Rainfall Intensity,0
Rainfall F=5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=5,Rainfall Intensity,0
Rainfall F=2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=2,Rainfall Intensity,0
Rainfall F=1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=1,Rainfall Intensity,0
Rainfall F=0.5,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.5,Rainfall Intensity,0
Rainfall F=0.2,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.2,Rainfall Intensity,0
Rainfall F=0.1,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.1,Rainfall Intensity,0
Rainfall F=0.05,3,5,1,C:\dev\mikepluspy\tests\testdata\Db\Sirius\Sir...,F=0.05,Rainfall Intensity,1


#### Run each event

In [15]:
# One existing simulation will be used
db.tables.msm_Project.select().to_dataframe()

,MUID,Sqn,IncludeToBatchNo,ScenarioName,EventID,ActiveProject,Enable_Catchment,Enable_CS,Enable_River,Enable_2DOverland,...,DecouplingPath,ELIntegrationNo,ELUpdateFrequency,M2DADDtMax,M2DADDtMin,M2DADMaxCFL,MJLFileName,HDOutputNo,HDFolderPath,SummaryID
Sirius_1_DEMO,Sirius_1_DEMO,1,1,Base,0,1,0,1,0,0,...,None,3,1,10.0,0.01,0.8,None,1,None,Summary_Sirius_1


In [16]:
# Get the scenarios with "Rainfall" in the name
scenarios = [scenario for scenario in db.scenarios if "Rainfall" in scenario.name]
scenarios

[Scenario <Rainfall  F=20>,
 Scenario <Rainfall  F=10>,
 Scenario <Rainfall  F=5>,
 Scenario <Rainfall  F=2>,
 Scenario <Rainfall  F=1>,
 Scenario <Rainfall  F=0.5>,
 Scenario <Rainfall  F=0.2>,
 Scenario <Rainfall  F=0.1>,
 Scenario <Rainfall  F=0.05>]

In [17]:
# Run each scenario
results = {}

for scenario in scenarios:
    updated_muids = (
        db.tables.msm_Project
        .update({
            db.tables.msm_Project.columns.ScenarioName : scenario.name
        })
        .where(f"MUID = 'Sirius_1_DEMO'")
        .execute()
    )

    if not updated_muids:
        raise ValueError(f"Failed to update scenario '{scenario.name}'.")

    result_files = db.run(simulation_muid="Sirius_1_DEMO")
    result_files = [f.name for f in result_files]           # hide full path (optional)
    results[scenario.name] = result_files

results

{'Rainfall  F=20': ['Sirius_1_DEMORainfall  F=20Default_Network_HD.res1d'],
 'Rainfall  F=10': ['Sirius_1_DEMORainfall  F=10Default_Network_HD.res1d'],
 'Rainfall  F=5': ['Sirius_1_DEMORainfall  F=5Default_Network_HD.res1d'],
 'Rainfall  F=2': ['Sirius_1_DEMORainfall  F=2Default_Network_HD.res1d'],
 'Rainfall  F=1': ['Sirius_1_DEMORainfall  F=1Default_Network_HD.res1d'],
 'Rainfall  F=0.5': ['Sirius_1_DEMORainfall  F=0.5Default_Network_HD.res1d'],
 'Rainfall  F=0.2': ['Sirius_1_DEMORainfall  F=0.2Default_Network_HD.res1d'],
 'Rainfall  F=0.1': ['Sirius_1_DEMORainfall  F=0.1Default_Network_HD.res1d'],
 'Rainfall  F=0.05': ['Sirius_1_DEMORainfall  F=0.05Default_Network_HD.res1d']}

In [18]:
db.close()